In [1]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from math import ceil, floor, trunc

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
con_lite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('max_rows',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 2, 22)

### Today = Last Closed Business Day

In [2]:
#today = today - timedelta(days=3)
today

datetime.date(2022, 2, 22)

### Restart and run all cells

In [3]:
def number_of_spread(x):
    maxp = x[1]
    minp = x[5]
    div = 0.0
    gap = 0.0
    spd = 0
    spd1 = 0
    spd2 = 0
    if (maxp > minp): 
        mult = 1
    else:
    	tmpp = minp
    	minp = maxp
    	maxp = tmpp
    	mult = -1
        
    gap = round(maxp - minp,2)   
    
    if (maxp > 400.00):
        div = 2.0
        boundary = div * 200
        if ((minp < boundary)):
            spd1 = trunc((boundary - minp) / 1.00)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 200.00):
        div = 1.0
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.50)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 100.00):
        div = 0.50
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.25)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 25.00):
        div = 0.25
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.10)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div)
    elif (maxp > 10.00):
        div = 0.10
        boundary = div * 100
        if (minp < boundary):
            spd1 = ceil((boundary - minp) / 0.05)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            div = 0.10            
            spd = ceil(gap / div)
    elif (maxp > 5.00):
        div = 0.05
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.02)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2           
        else:
            div = 0.05            
            spd = ceil(gap / div)
    elif (maxp > 2.00):
        div = 0.02
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.01)
            spd2 = trunc((maxp - boundary) / div)
            spd = spd1 + spd2
        else:          
            spd = (gap / div)
    else:
        div = 0.01
        boundary = div * 100        
        spd = trunc(gap / div) 
        
    return spd * mult

In [4]:
cols = 'name spd target bl qty fm_price to_price ttl min min52 css_price status amount fm_date to_date days'.split()

In [5]:
format_dict = {
               'fm_price':'{:.2f}','to_price':'{:.2f}','price':'{:.2f}',    
               'max_price':'{:.2f}','min_price':'{:.2f}',
               'target':'{:.2f}','max':'{:.2f}','max52':'{:.2f}',
               'min':'{:.2f}','min52':'{:.2f}',
               'css_price':'{:.2f}','amount':'{:,.2f}',  
               'unit_cost':'{:.2f}','cost_amt':'{:,.2f}',
               'dividend':'{:.2f}','dividend_amount':'{:,.2f}',    
               'cost_amt':'{:,.2f}','div_amt':'{:,.2f}','cost_pct':'{:.2f}', 
               'buy_target':'{:.2f}','sell_target':'{:.2f}',
               'volume':'{:,.2f}','beta':'{:.2f}','cost':'{:,.2f}',
               'qty':'{:,}','available_qty':'{:,}','shares':'{:,}'}
pd.set_option("display.max_rows", None)

In [6]:
sql = '''
SELECT T.name,buy_target AS target, bl, T.qty,
fm_price,to_price,  ttl_spread AS ttl, A.min_price AS min, 
T.min_price AS min52, consensus_price AS css_price, buy_target * T.qty AS amount, T.status,
FM_date, to_date, days
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('B','O')
AND buy_target > 0
AND to_date = '%s'
'''
sql = sql % today
print(sql)

df_stock = pd.read_sql(sql, con_lite)
df_stock.sort_values(['name'],ascending=[True]).style.format(format_dict)


SELECT T.name,buy_target AS target, bl, T.qty,
fm_price,to_price,  ttl_spread AS ttl, A.min_price AS min, 
T.min_price AS min52, consensus_price AS css_price, buy_target * T.qty AS amount, T.status,
FM_date, to_date, days
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('B','O')
AND buy_target > 0
AND to_date = '2022-02-22'



,name,target,bl,qty,fm_price,to_price,ttl,min,min52,css_price,amount,status,fm_date,to_date,days
10,ASK,42.00,0,"3,000",44.50,46.00,6,1.12,18.25,53.67,"126,000.00",O,2022-02-18,2022-02-22,2
0,ASP,3.72,0,"30,000",3.78,3.88,5,2.12,2.12,4.40,"111,600.00",O,2022-02-14,2022-02-22,6
1,BGC,10.00,0,"10,000",10.50,10.30,-2,-0.95,9.20,11.90,"100,000.00",O,2022-02-17,2022-02-22,3
12,KKP,68.00,0,"3,000",68.75,69.75,4,67.50,49.75,78.42,"204,000.00",O,2022-02-21,2022-02-22,1
2,MAKRO,42.00,0,"3,000",41.75,43.25,6,3.59,35.00,50.10,"126,000.00",O,2022-02-17,2022-02-22,3
8,MEGA,40.50,1,"3,000",43.50,42.75,-3,-0.57,32.25,55.75,"121,500.00",B,2022-02-18,2022-02-22,2
9,NOBLE,5.85,0,"21,000",6.25,6.20,-1,-0.80,5.90,7.30,"122,850.00",B,2022-02-18,2022-02-22,2
3,PTTGC,54.25,3,"1,200",58.25,54.75,-14,-5.58,54.25,68.11,"65,100.00",B,2022-02-09,2022-02-22,9
4,ROJNA,6.30,0,"18,000",7.00,6.65,-7,-5.00,4.66,8.05,"113,400.00",O,2022-02-17,2022-02-22,3
14,SCCC,173.00,0,900,175.00,174.00,-2,173.00,152.00,196.10,"155,700.00",O,2022-02-21,2022-02-22,1


In [7]:
my_array = df_stock.to_numpy()
my_array

array([['ASP', 3.72, 0, 30000, 3.78, 3.88, 5, 2.12, 2.12, 4.4, 111600.0,
        'O', '2022-02-14', '2022-02-22', 6],
       ['BGC', 10.0, 0, 10000, 10.5, 10.3, -2, -0.95, 9.2, 11.9,
        100000.0, 'O', '2022-02-17', '2022-02-22', 3],
       ['MAKRO', 42.0, 0, 3000, 41.75, 43.25, 6, 3.59, 35.0, 50.1,
        126000.0, 'O', '2022-02-17', '2022-02-22', 3],
       ['PTTGC', 54.25, 3, 1200, 58.25, 54.75, -14, -5.58, 54.25, 68.11,
        65100.0, 'B', '2022-02-09', '2022-02-22', 9],
       ['ROJNA', 6.3, 0, 18000, 7.0, 6.65, -7, -5.0, 4.66, 8.05,
        113400.0, 'O', '2022-02-17', '2022-02-22', 3],
       ['SPALI', 21.6, 0, 9000, 23.4, 22.4, -10, -4.27, 19.0, 25.77,
        194400.0, 'O', '2022-02-15', '2022-02-22', 5],
       ['TMT', 9.8, 0, 15000, 10.2, 10.8, 6, 5.88, 7.2, 11.33, 147000.0,
        'B', '2022-02-09', '2022-02-22', 9],
       ['SINGER', 39.25, 4, 1500, 42.25, 44.5, 9, 2.37, 28.25, 56.43,
        58875.0, 'B', '2022-02-18', '2022-02-22', 2],
       ['MEGA', 40.5, 1, 30

In [8]:
tmp = (np.apply_along_axis(number_of_spread, axis=1, arr=my_array))
tmp

array([ -8.,  -3.,  -5.,  -2.,  -7.,  -8., -12., -21.,  -9.,  -7., -16.,
        -2.,  -7.,  -9.,  -2.,  -5.])

In [9]:
df_stock['spd'] = tmp.tolist()
df_stock['spd'] = df_stock['spd'].astype(int)
df_stock.style.format(format_dict)

,name,target,bl,qty,fm_price,to_price,ttl,min,min52,css_price,amount,status,fm_date,to_date,days,spd
0,ASP,3.72,0,"30,000",3.78,3.88,5,2.12,2.12,4.40,"111,600.00",O,2022-02-14,2022-02-22,6,-8
1,BGC,10.00,0,"10,000",10.50,10.30,-2,-0.95,9.20,11.90,"100,000.00",O,2022-02-17,2022-02-22,3,-3
2,MAKRO,42.00,0,"3,000",41.75,43.25,6,3.59,35.00,50.10,"126,000.00",O,2022-02-17,2022-02-22,3,-5
3,PTTGC,54.25,3,"1,200",58.25,54.75,-14,-5.58,54.25,68.11,"65,100.00",B,2022-02-09,2022-02-22,9,-2
4,ROJNA,6.30,0,"18,000",7.00,6.65,-7,-5.00,4.66,8.05,"113,400.00",O,2022-02-17,2022-02-22,3,-7
5,SPALI,21.60,0,"9,000",23.40,22.40,-10,-4.27,19.00,25.77,"194,400.00",O,2022-02-15,2022-02-22,5,-8
6,TMT,9.80,0,"15,000",10.20,10.80,6,5.88,7.20,11.33,"147,000.00",B,2022-02-09,2022-02-22,9,-12
7,SINGER,39.25,4,"1,500",42.25,44.50,9,2.37,28.25,56.43,"58,875.00",B,2022-02-18,2022-02-22,2,-21
8,MEGA,40.50,1,"3,000",43.50,42.75,-3,-0.57,32.25,55.75,"121,500.00",B,2022-02-18,2022-02-22,2,-9
9,NOBLE,5.85,0,"21,000",6.25,6.20,-1,-0.80,5.90,7.30,"122,850.00",B,2022-02-18,2022-02-22,2,-7


In [10]:
df_stock[cols].style.format(format_dict)

,name,spd,target,bl,qty,fm_price,to_price,ttl,min,min52,css_price,status,amount,fm_date,to_date,days
0,ASP,-8,3.72,0,"30,000",3.78,3.88,5,2.12,2.12,4.40,O,"111,600.00",2022-02-14,2022-02-22,6
1,BGC,-3,10.00,0,"10,000",10.50,10.30,-2,-0.95,9.20,11.90,O,"100,000.00",2022-02-17,2022-02-22,3
2,MAKRO,-5,42.00,0,"3,000",41.75,43.25,6,3.59,35.00,50.10,O,"126,000.00",2022-02-17,2022-02-22,3
3,PTTGC,-2,54.25,3,"1,200",58.25,54.75,-14,-5.58,54.25,68.11,B,"65,100.00",2022-02-09,2022-02-22,9
4,ROJNA,-7,6.30,0,"18,000",7.00,6.65,-7,-5.00,4.66,8.05,O,"113,400.00",2022-02-17,2022-02-22,3
5,SPALI,-8,21.60,0,"9,000",23.40,22.40,-10,-4.27,19.00,25.77,O,"194,400.00",2022-02-15,2022-02-22,5
6,TMT,-12,9.80,0,"15,000",10.20,10.80,6,5.88,7.20,11.33,B,"147,000.00",2022-02-09,2022-02-22,9
7,SINGER,-21,39.25,4,"1,500",42.25,44.50,9,2.37,28.25,56.43,B,"58,875.00",2022-02-18,2022-02-22,2
8,MEGA,-9,40.50,1,"3,000",43.50,42.75,-3,-0.57,32.25,55.75,B,"121,500.00",2022-02-18,2022-02-22,2
9,NOBLE,-7,5.85,0,"21,000",6.25,6.20,-1,-0.80,5.90,7.30,B,"122,850.00",2022-02-18,2022-02-22,2


In [11]:
buy_alert = df_stock[cols]
buy_alert.set_index(['name'],inplace=True)
buy_alert.sort_values(['spd'],ascending=[False]).style.format(format_dict)

,spd,target,bl,qty,fm_price,to_price,ttl,min,min52,css_price,status,amount,fm_date,to_date,days
name,,,,,,,,,,,,,,,
PTTGC,-2,54.25,3,"1,200",58.25,54.75,-14,-5.58,54.25,68.11,B,"65,100.00",2022-02-09,2022-02-22,9
TQM,-2,45.00,6,"4,500",45.25,45.50,1,44.75,44.50,91.94,O,"202,500.00",2022-02-21,2022-02-22,1
SCCC,-2,173.00,0,900,175.00,174.00,-2,173.00,152.00,196.10,O,"155,700.00",2022-02-21,2022-02-22,1
BGC,-3,10.00,0,"10,000",10.50,10.30,-2,-0.95,9.20,11.90,O,"100,000.00",2022-02-17,2022-02-22,3
MAKRO,-5,42.00,0,"3,000",41.75,43.25,6,3.59,35.00,50.10,O,"126,000.00",2022-02-17,2022-02-22,3
SNC,-5,16.50,0,"7,500",17.80,17.00,-8,16.80,12.50,20.75,O,"123,750.00",2022-02-21,2022-02-22,1
ROJNA,-7,6.30,0,"18,000",7.00,6.65,-7,-5.00,4.66,8.05,O,"113,400.00",2022-02-17,2022-02-22,3
NOBLE,-7,5.85,0,"21,000",6.25,6.20,-1,-0.80,5.90,7.30,B,"122,850.00",2022-02-18,2022-02-22,2
KKP,-7,68.00,0,"3,000",68.75,69.75,4,67.50,49.75,78.42,O,"204,000.00",2022-02-21,2022-02-22,1


In [12]:
file_name = 'buy-alert.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

buy_alert.sort_values(['spd','name'],ascending=[False,True]).to_csv(output_file)
buy_alert.sort_values(['spd','name'],ascending=[False,True]).to_csv(data_file)
buy_alert.sort_values(['spd','name'],ascending=[False,True]).to_csv(box_file)
buy_alert.sort_values(['spd','name'],ascending=[False,True]).to_csv(one_file)